In [1]:
# Copyright 2021 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

In [2]:
# !sync; echo 3 > /proc/sys/vm/drop_caches

In [3]:
import os
from time import time
import re
import glob
import warnings

# tools for data preproc/loading
import torch
# torch.multiprocessing.set_start_method("spawn")
# import rmm
# import nvtabular as nvt
# from nvtabular.ops import Normalize,  Categorify,  LogOp, FillMissing, Clip, get_embedding_sizes
# from nvtabular.loader.torch import TorchAsyncItr, DLDataLoader
# from nvtabular.utils import device_mem_size

# tools for training

In [4]:
# define some information about where to get our data
INPUT_DATA_DIR = os.environ.get('INPUT_DATA_DIR', '/raid/criteo/tests/crit_int_pq')
OUTPUT_DATA_DIR = os.environ.get('OUTPUT_DATA_DIR', '/raid/criteo/tests/test_dask') # where we'll save our procesed data to
BATCH_SIZE = int(os.environ.get('BATCH_SIZE', 32768))
AMP = os.environ.get("AMP", "true") 
AMP = True if AMP.lower() in "true" else False
SHUFFLE = os.environ.get("SHUFFLE", False)
PARTS_PER_CHUNK = int(os.environ.get('PARTS_PER_CHUNK', 2))
NUM_TRAIN_DAYS = 23 # number of days worth of data to use for training, the rest will be used for validation

# define our dataset schema
CONTINUOUS_COLUMNS = ['I' + str(x) for x in range(1,14)]
CATEGORICAL_COLUMNS =  ['C' + str(x) for x in range(1,27)]
LABEL_COLUMNS = ['label']
COLUMNS = CONTINUOUS_COLUMNS + CATEGORICAL_COLUMNS + LABEL_COLUMNS

In [ ]:
output_train_dir = os.path.join(OUTPUT_DATA_DIR, 'train/')
output_valid_dir = os.path.join(OUTPUT_DATA_DIR, 'valid/')
! mkdir -p $output_train_dir
! mkdir -p $output_valid_dir

In [ ]:
train_paths = glob.glob(os.path.join(output_train_dir, "*.parquet"))
valid_paths = glob.glob(os.path.join(output_valid_dir, "*.parquet"))

from pyarrow.parquet import read_metadata
train_stats = read_metadata(train_paths[0])
valid_stats = read_metadata(valid_paths[0])
print(train_stats, valid_stats)

In [ ]:
def batch_transform(batch, cont_cols=CONTINUOUS_COLUMNS, cat_cols=CATEGORICAL_COLUMNS, label_cols=LABEL_COLUMNS):
    x_cat = torch.tensor(batch[sorted(cat_cols)].values,dtype=torch.int64)
    x_cont = torch.tensor(batch[cont_cols].values, dtype=torch.float32)
    y = torch.tensor(batch[label_cols[0]], dtype=torch.float32)
    return x_cat, x_cont, y

In [ ]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader
# Create Dataset
class GenericDataset(Dataset):
    def __init__(self, tar_file, transform=None):
        self.frame = pd.read_parquet(tar_file)[:10000000]
        self.transform = transform
    
    def __len__(self):
        return self.frame.shape[0]
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        rec = self.frame.iloc[idx]
        if self.transform:
            rec = self.transform(rec)
        re
        return rec

In [ ]:
device = torch.device("cpu")

In [ ]:
train_loader = DataLoader(GenericDataset(train_paths[0], transform=batch_transform), 
                          pin_memory=True,
                          batch_size=BATCH_SIZE,
                          num_workers=20,
#                           collate_fn=collate,
                          shuffle=SHUFFLE,
                         )
valid_loader = DataLoader(GenericDataset(valid_paths[0], transform=batch_transform), 
                          pin_memory=True, 
                          batch_size=BATCH_SIZE,
                          num_workers=20,
#                           collate_fn=collate,
                          shuffle=SHUFFLE,
                         )

In [ ]:
EMBEDDING_DROPOUT_RATE = 0.04
DROPOUT_RATES = [0.001, 0.01]
HIDDEN_DIMS = [1000, 500]
LEARNING_RATE = 0.001
EPOCHS = 1
embeddings = {'C1': (7599500, 16),
 'C10': (5345303, 16),
 'C11': (561810, 16),
 'C12': (242827, 16),
 'C13': (11, 6),
 'C14': (2209, 16),
 'C15': (10616, 16),
 'C16': (100, 16),
 'C17': (4, 3),
 'C18': (968, 16),
 'C19': (15, 7),
 'C2': (33521, 16),
 'C20': (7838519, 16),
 'C21': (2580502, 16),
 'C22': (6878028, 16),
 'C23': (298771, 16),
 'C24': (11951, 16),
 'C25': (97, 16),
 'C26': (35, 12),
 'C3': (17022, 16),
 'C4': (7339, 16),
 'C5': (20046, 16),
 'C6': (4, 3),
 'C7': (7068, 16),
 'C8': (1377, 16),
 'C9': (63, 16)}

In [ ]:
from nvtabular.framework_utils.torch.models import Model
from nvtabular.framework_utils.torch.utils import process_epoch
model = Model(
    embedding_table_shapes=embeddings,
    num_continuous=len(CONTINUOUS_COLUMNS),
    emb_dropout=EMBEDDING_DROPOUT_RATE,
    layer_hidden_dims=HIDDEN_DIMS,
    layer_dropout_rates=DROPOUT_RATES,
).to(device)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
def rmspe_func(y_pred, y):
    "Return y_pred and y to non-log space and compute RMSPE"
    y_pred, y = torch.exp(y_pred) - 1, torch.exp(y) - 1
    pct_var = (y_pred - y) / y
    return (pct_var**2).mean().pow(0.5)

In [ ]:
for epoch in range(EPOCHS):
    start_train=time()
    train_loss, y_pred, y = process_epoch(train_loader, 
                                          model, 
                                          train=True, 
                                          optimizer=optimizer,
                                          #transform=batch_transform,
                                          amp=AMP,
                                          #device=device,
                                         )
    train_rmspe = rmspe_func(y_pred, y)
    train_time=time() - start_train
    y_pred = None
    y = None
    print(train_time)
    start_valid=time()
    valid_loss, y_pred, y = process_epoch(valid_loader,
                                          model, 
                                          train=False, 
                                          optimizer=optimizer,
                                          #transform=batch_transform,
                                          amp=AMP,
                                          #device=device,
                                         )
    valid_rmspe = rmspe_func(y_pred, y)
    valid_time = time() - start_valid
    y_pred = None
    y = None
    print(f"Train:{train_time} + Valid:{valid_time} = EpochTotal:{train_time + valid_time}")